In [12]:
import requests
from bs4 import BeautifulSoup
from pprint import pprint
import pandas as pd

In [13]:
URL = "https://www.tinkoff.ru/invest/stocks/?country=All&orderType=Asc&sortType=ByName&start=0&end=12"
HEADERS = {'user-agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36" }

In [14]:
def get_HTML(URL, params = None):
    r = requests.get(URL, headers = HEADERS, params = params)
    return r

In [60]:
def parse(URL):
    html = get_HTML(URL)
    if html.status_code == 200:
        soup = BeautifulSoup(html.text, "html.parser")
        items = soup.find_all("a", class_ = "Link-module__link__Wv1b Link-module__link_theme_default_gf5wq")
        
        title_list = pd.Series([item.find('div', class_ = "Caption__caption_cyYZT") \
                         for item in items if item.find('div', class_ = "Caption__caption_cyYZT") ]).apply(lambda i: i.text)
        ticker_list = pd.Series([item.find('div', class_ = "Caption__subcaption_xTAKS") \
                         for item in items if item.find('div', class_ = "Caption__subcaption_xTAKS") ]).apply(lambda i: i.text)
    else:
        print("Error")
    return [title_list ,  ticker_list]


In [56]:
URL_all = "https://www.tinkoff.ru/invest/stocks/?country=All&orderType=Asc&sortType=ByName&start=0&end=12"

In [61]:
parse(URL_all)

[0       1-800-Flowers.com Inc
 1            10X Genomics Inc
 2            1Life Healthcare
 3                2seventy bio
 4                      2U Inc
 5             3D Systems Corp
 6                  3M Company
 7                       51job
 8                     8x8 Inc
 9     A. O. Smith Corporation
 10                       AAON
 11                   AAR Corp
 dtype: object,
 0     FLWS
 1      TXG
 2     ONEM
 3     TSVT
 4     TWOU
 5      DDD
 6      MMM
 7     JOBS
 8     EGHT
 9      AOS
 10    AAON
 11     AIR
 dtype: object]

In [18]:
all_URL = []
for i in range(1,6):
    all_URL.append("https://www.tinkoff.ru/invest/stocks/?country=All&orderType=Asc&sortType=ByName&start=0&end=" + str(12*i))

In [41]:
full_title_list

[0       1-800-Flowers.com Inc
 1            10X Genomics Inc
 2            1Life Healthcare
 3                2seventy bio
 4                      2U Inc
 5             3D Systems Corp
 6                  3M Company
 7                       51job
 8                     8x8 Inc
 9     A. O. Smith Corporation
 10                       AAON
 11                   AAR Corp
 dtype: object,
 0     FLWS
 1      TXG
 2     ONEM
 3     TSVT
 4     TWOU
 5      DDD
 6      MMM
 7     JOBS
 8     EGHT
 9      AOS
 10    AAON
 11     AIR
 dtype: object]